In [1]:
#!pip install  pyspark==3.3.1.

In [2]:
from pyspark.sql import  SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_trasformation")\
    .getOrCreate()

25/01/30 10:56:31 WARN Utils: Your hostname, maxweber-VM resolves to a loopback address: 127.0.1.1; using 172.19.181.10 instead (on interface eth0)
25/01/30 10:56:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/30 10:56:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.json("/home/maxweber/Notbook/airflow/datalake")

In [5]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{38, 37, 2025-01...|{[{2025-01-29T18:...|{1234567890abcdef}|  2025-01-30|
|[{66, 27, 2025-01...|{[{2025-01-29T08:...|{1234567890abcdef}|  2025-01-30|
|[{47, 44, 2025-01...|{[{2025-01-29T10:...|{1234567890abcdef}|  2025-01-30|
|[{53, 93, 2025-01...|{[{2025-01-29T16:...|              null|  2025-01-30|
|[{12, 41, 2025-01...|{[{2025-01-26T11:...|{1234567890abcdef}|  2025-01-26|
|[{79, 19, 2025-01...|{[{2025-01-26T03:...|              null|  2025-01-26|
|[{33, 69, 2025-01...|{[{2025-01-25T12:...|{1234567890abcdef}|  2025-01-25|
|[{50, 24, 2025-01...|{[{2025-01-25T15:...|              null|  2025-01-25|
|[{74, 95, 2025-01...|{[{2025-01-24T13:...|              null|  2025-01-24|
+--------------------+--------------------+------------------+------------+



In [8]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [9]:
from pyspark.sql import functions as f

In [10]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [11]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{38, 37, 2025-01-...|
|{43, 20, 2025-01-...|
|{78, 23, 2025-01-...|
|{61, 54, 2025-01-...|
|{18, 75, 2025-01-...|
|{50, 86, 2025-01-...|
|{0, 87, 2025-01-2...|
|{63, 88, 2025-01-...|
|{19, 43, 2025-01-...|
|{61, 9, 2025-01-2...|
|{66, 27, 2025-01-...|
|{35, 89, 2025-01-...|
|{31, 24, 2025-01-...|
|{42, 89, 2025-01-...|
|{82, 14, 2025-01-...|
|{91, 24, 2025-01-...|
|{10, 5, 2025-01-2...|
|{92, 85, 2025-01-...|
|{90, 96, 2025-01-...|
|{39, 47, 2025-01-...|
+--------------------+
only showing top 20 rows



In [14]:
df.select(f.explode("data").alias("tweets"))\
    .select("tweets.author_id","tweets.conversation_id",
            "tweets.created_at","tweets.id",
            "tweets.public_metrics.*","tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [17]:
tweet_df=df.select(f.explode("data").alias("tweets"))\
    .select("tweets.author_id","tweets.conversation_id",
            "tweets.created_at","tweets.id",
            "tweets.public_metrics.*","tweets.text")   # O * no campo 'tweets.public_metrics.*' tras todas o campos abaixo dele como coluna

In [19]:
tweet_df.show()

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       38|             37|2025-01-29T13:04:...| 37|        35|         93|         62|           57|Outro tweet fictí...|
|       43|             20|2025-01-29T16:22:...| 28|        18|         48|         23|           29|Outro tweet fictí...|
|       78|             23|2025-01-29T01:52:...| 50|         7|         89|         59|            1|Tweet fictício ge...|
|       61|             54|2025-01-29T19:15:...| 56|        50|         75|         85|           74|Este é um tweet f...|
|       18|             75|2025-01-29T09:34:...| 60|        56|         64|        100|           27|Este é um tweet f...|
|       50|     

In [22]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [31]:
df.select(f.explode("includes.users").alias("users")).select("users.created_at","users.id","users.name","users.username").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [35]:
users_df=df.select(f.explode("includes.users").alias("users"))\
    .select("users.created_at","users.id","users.name","users.username")

In [36]:
users_df.show()

+--------------------+---+-------+--------+
|          created_at| id|   name|username|
+--------------------+---+-------+--------+
|2025-01-29T18:17:...| 44| User 1|   user1|
|2025-01-29T02:00:...|  0| User 2|   user2|
|2025-01-29T12:29:...|  2| User 3|   user3|
|2025-01-29T03:47:...| 29| User 4|   user4|
|2025-01-29T08:56:...| 56| User 5|   user5|
|2025-01-29T12:23:...| 19| User 6|   user6|
|2025-01-29T09:42:...|  9| User 7|   user7|
|2025-01-29T04:57:...| 50| User 8|   user8|
|2025-01-29T10:56:...| 53| User 9|   user9|
|2025-01-29T05:18:...| 16|User 10|  user10|
|2025-01-29T08:15:...| 76| User 1|   user1|
|2025-01-29T22:49:...| 99| User 2|   user2|
|2025-01-29T20:45:...| 55| User 3|   user3|
|2025-01-29T05:42:...| 92| User 4|   user4|
|2025-01-29T12:15:...| 14| User 5|   user5|
|2025-01-29T18:09:...| 96| User 6|   user6|
|2025-01-29T07:38:...| 63| User 7|   user7|
|2025-01-29T17:47:...| 54| User 8|   user8|
|2025-01-29T20:41:...| 85| User 9|   user9|
|2025-01-29T11:50:...| 33|User 1

In [38]:
tweet_df.coalesce(1).write.mode("overwrite").json("output/tweet")
users_df.coalesce(1).write.mode("overwrite").json("output/user")